In [1]:
# %cd /content/drive/MyDrive/Colab Notebooks/fastai-practice

# !git config --global user.email 'intaek428@gmail.com'
# !git config --global user.name 'ITHwang'

# !git add .
# !git commit -m ''
# !git push

!pip install -Uqq fastbook
from fastai.text.all import *

In [2]:
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup')]

In [14]:
dls = DataBlock(
    blocks=(TextBlock.from_folder(path), CategoryBlock),
    get_y=parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

In [15]:
files = get_text_files(path, folders=['train', 'test'])
txts = L(o.open().read() for o in files[:2000])

In [20]:
tok = Tokenizer.from_folder(path)
tok.path

Path('/root/.fastai/data/imdb')

In [21]:
tok.output_dir

Path('/root/.fastai/data/imdb_tok')

In [23]:
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#148) ['xxbos','xxmaj','i',"'m",'not','sure','why','i','disliked','this'...]

In [24]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:10]

TensorText([   2,    8,   18,  155,   37,  291,  143,   18, 4523,   20])

In [25]:
nums_dec = num.decode(nums[0][:10])
nums_dec

(#10) ['xxbos','xxmaj','i',"'m",'not','sure','why','i','disliked','this']

In [30]:
def f(x:int): return x + 1
tfm = Transform(f)
tfm(2),tfm(2.0)

(3, 2.0)

In [31]:
@Transform
def f(x:int): return x+1
f(2),f(2.0)

(3, 2.0)

In [35]:
class NormalizeMean(Transform):
    def setups(self, items): self.mean = sum(items) / len(items)
    def encodes(self, x): return x - self.mean
    def decodes(self, x): return x + self.mean

In [36]:
tfm = NormalizeMean()
tfm.setup([1,2,3,4,5])
start = 2
y = tfm(start)
z = tfm.decode(y)
tfm.mean, y, z

(3.0, -1.0, 2.0)

In [37]:
tfms = Pipeline([tok, num])
t = tfms(txts[0])
t[:20]

TensorText([   2,    8,   18,  155,   37,  291,  143,   18, 4523,   20,   33,   48,
          97,   10,    8,  236,    8,   18,  155,  110])

In [38]:
tfms.decode(t)[:100]

"xxbos xxmaj i 'm not sure why i disliked this film so much . xxmaj maybe xxmaj i 'm too old or too m"